In [2]:
import unicodedata

def segment_gujarati_paragraph(paragraph):
    # Function to segment a Gujarati paragraph into characters and generate a dataset.

    # Initialize an empty list to store character-label pairs
    char_dataset = []

    # Loop through each character in the paragraph
    for char in paragraph:
        # Skip whitespace characters
        if unicodedata.category(char).startswith('Z'):
            continue

        # Get the Unicode code point of the character
        char_code = ord(char)

        # Append the character and its code point to the dataset
        char_dataset.append((char, char_code))

    return char_dataset

# Example Gujarati paragraph (replace this with your own data)
gujarati_paragraph = "ગુજરાતી"

# Segment the paragraph into characters and generate the dataset
dataset = segment_gujarati_paragraph(gujarati_paragraph)

# Print the dataset
for char, char_code in dataset:
    print(f"Character: {char}, Unicode Code Point: {char_code}")


Character: ગ, Unicode Code Point: 2711
Character: ુ, Unicode Code Point: 2753
Character: જ, Unicode Code Point: 2716
Character: ર, Unicode Code Point: 2736
Character: ા, Unicode Code Point: 2750
Character: ત, Unicode Code Point: 2724
Character: ી, Unicode Code Point: 2752


In [3]:
# import pytesseract
# from PIL import Image

# # Path to the Tesseract executable (if not in your system PATH)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'  # Update this path if necessary

# # Load the image
# image_path = 'img1.tif'  # Replace with the path to your image
# img = Image.open(image_path)

# # Perform OCR on the image to extract Gujarati text
# print(pytesseract.get_languages(config=''))
# text = pytesseract.image_to_string(img)

# # Print the extracted text
# print(text)

In [ ]:
# import pytesseract

# # Test Tesseract installation
# print(pytesseract.get_tesseract_version())

In [5]:
# pip show pytesseract-ocr-guj

In [4]:
# pip install pytesseract-ocr-guj

# DATASET

# SPLIT LINE

In [11]:
from PIL import Image, ImageStat
import os
import time

# Load the larger image
large_image_path = 'INPUT/test8.jpg'  # Replace with the path to your larger image
large_image = Image.open(large_image_path)

# Define the size of each smaller image (100x100 pixels in this example)
h = (int)(large_image.height / 14)
w = (int)(large_image.width / 10)
small_image_size = (w, h)

# Calculate the number of smaller images to create
num_rows = large_image.height // small_image_size[1]
num_cols = large_image.width // small_image_size[0]

# Create a blank image to compare with (adjust the color as needed)
blank_color = (255, 255, 255)  # White
blank_image = Image.new('RGB', small_image_size, blank_color)

# Create a directory to store the smaller images
output_directory = 'small_images'
os.makedirs(output_directory, exist_ok=True)

# Get the current timestamp to make filenames unique
timestamp = int(time.time())

# Loop through the larger image and create and save only non-blank smaller images
for row in range(num_rows):
    for col in range(num_cols):
        # Define the cropping coordinates for the current smaller image
        left = col * small_image_size[0]
        upper = row * small_image_size[1]
        right = left + small_image_size[0]
        lower = upper + small_image_size[1]

        # Crop the smaller image
        small_image = large_image.crop((left, upper, right, lower))

        # Convert the cropped image to grayscale
        small_image_gray = small_image.convert('L')

        # Calculate the variance of pixel values in the grayscale image
        stat = ImageStat.Stat(small_image_gray)
        pixel_variance = stat.var[0]

        # If the pixel variance is below a threshold, consider it blank (adjust the threshold as needed)
        blank_threshold = 25  # Adjust as needed
        if pixel_variance < blank_threshold:
            continue  # Skip saving if mostly blank

        # Generate a unique filename with a timestamp
        small_image_path = os.path.join(output_directory, f'small_image_{timestamp}_{row}_{col}.png')
        small_image.save(small_image_path)

print(f"{num_rows * num_cols} smaller images saved to the 'small_images' directory, excluding mostly blank images.")


140 smaller images saved to the 'small_images' directory, excluding mostly blank images.


# BOUNDING BOX

In [12]:
import cv2
import numpy as np
import os

# Function to extract the region containing text
def extract_text_region(image_path):
    # Your existing code here...
    # Read the input image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Use Canny edge detection to find edges in the image
    edges = cv2.Canny(blurred, 50, 150)
    
    # Find contours in the edges image
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize bounding box coordinates
    x_min, x_max, y_min, y_max = float('inf'), -1, float('inf'), -1
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x + w)
        y_max = max(y_max, y + h)
    
    # Extract the text region
    if x_min < x_max and y_min < y_max:
    # Extract the text region
        text_region = image[int(y_min):int(y_max), int(x_min):int(x_max)]
    else:
        text_region = None
    
    return text_region

# Input and output folder paths
input_folder = 'small_images'
output_folder = 'output_text_images'

    # Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.tif') or filename.endswith('.jpg') or filename.endswith('.png'):
        # Construct the full path of the input image
        input_image_path = os.path.join(input_folder, filename)

        # Extract the text region
        text_image = extract_text_region(input_image_path)

        # Construct the full path of the output image
        output_image_path = os.path.join(output_folder, filename)

        # Save the new image containing the text region
        if text_image is not None:
            cv2.imwrite(output_image_path, text_image)
            print(f'Text region extracted and saved for {filename}')
        else:
            print(f'No text found in {filename}')


Text region extracted and saved for small_image_1698345896_0_0.png
Text region extracted and saved for small_image_1698345896_0_1.png
No text found in small_image_1698345896_0_2.png
No text found in small_image_1698345896_0_3.png
No text found in small_image_1698345896_0_4.png
No text found in small_image_1698345896_0_5.png
No text found in small_image_1698345896_0_6.png
No text found in small_image_1698345896_0_7.png
No text found in small_image_1698345896_0_8.png
Text region extracted and saved for small_image_1698345896_10_0.png
Text region extracted and saved for small_image_1698345896_10_1.png
Text region extracted and saved for small_image_1698345896_10_2.png
Text region extracted and saved for small_image_1698345896_10_3.png
Text region extracted and saved for small_image_1698345896_10_4.png
Text region extracted and saved for small_image_1698345896_10_5.png
Text region extracted and saved for small_image_1698345896_10_6.png
Text region extracted and saved for small_image_169834

# RESIZE

In [13]:
import os
import cv2

# Set the folder path containing images to be resized
folder_path = r'C:\Users\Nilanjan Vyas\Documents\Minor_Project\output_text_images'

# Define the new width and height for the resized image
new_width = 32
new_height = 32

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Loop through each file in the folder
for file in files:
    # Check if the file is an image
    if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
        # Load the image
        image_path = os.path.join(folder_path, file)
        image = cv2.imread(image_path)

        # Resize the image
        resized_image = cv2.resize(image, (new_width, new_height))

        # Overwrite the existing image with the resized image
        cv2.imwrite(image_path, resized_image)

#         print(f'Resized and saved: {file}')

print('Resizing complete.')


Resizing complete.


# DATA AUGMENTATION

In [16]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Set the input folder path
train_folder = r'C:\Users\Nilanjan Vyas\Documents\Minor_Project\Dataset\Test'

# Initialize the ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.1,
    fill_mode='nearest')

# Get a list of all subfolders in the train folder
subfolders = [f.path for f in os.scandir(train_folder) if f.is_dir()]

# Loop through each subfolder
for folder in subfolders:
    # Get a list of all files in the subfolder
    files = os.listdir(folder)

    # Loop through each file in the subfolder
    for file in files:
        # Check if the file is an image
        if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
            # Load the image
            image_path = os.path.join(folder, file)
            image = np.array(Image.open(image_path))
            image = image.reshape((1,) + image.shape)  # Reshape the image for the flow method

            # Generate augmented images and save in the same folder
            i = 0
            for batch in datagen.flow(image, batch_size=1, save_to_dir=folder, save_prefix='aug', save_format='jpeg'):
                i += 1
                if i >= 10:  # Generate 10 augmented images for each original image
                    break

#             print(f'Augmented and saved images for: {file} in folder: {folder}')

print('Augmentation complete.')


Augmentation complete.


In [2]:
# TEST1
# import os
# import numpy as np
# from keras.preprocessing.image import ImageDataGenerator
# from PIL import Image

# # Set the input and output folder paths
# input_folder = r'C:\Users\Nilanjan Vyas\Documents\Minor_Project\output_text_images'
# output_folder = r'C:\Users\Nilanjan Vyas\Documents\Minor_Project\augmented_images'

# # Ensure the output folder exists, if not, create it
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # Initialize the ImageDataGenerator for augmentation
# datagen = ImageDataGenerator(
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=False,
#     fill_mode='nearest')

# # Get a list of all files in the input folder
# files = os.listdir(input_folder)

# # Loop through each file in the input folder
# for file in files:
#     # Check if the file is an image
#     if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
#         # Load the image
#         image = np.array(Image.open(image_path))
#         image = image.reshape((1,) + image.shape)  # Reshape the image for the flow method

#         # Generate augmented images
#         i = 0
#         for batch in datagen.flow(image, batch_size=1, save_to_dir=output_folder, save_prefix=file.split('.')[0], save_format='jpeg'):
#             i += 1
#             if i >= 10:  # Generate 10 augmented images for each original image
#                 break

# #         print(f'Augmented and saved images for: {file}')

# print('Augmentation complete.')


Augmentation complete.


In [30]:
# TEST2
# import os
# import numpy as np
# from keras.preprocessing.image import ImageDataGenerator
# from PIL import Image

# # Set the input and output folder paths
# input_folder = r'C:\Users\Nilanjan Vyas\Documents\Minor_Project\Dataset\Train\ba'
# # output_folder = 'path/to/output/folder'

# # Ensure the output folder exists, if not, create it
# # if not os.path.exists(output_folder):
# #     os.makedirs(output_folder)

# # Initialize the ImageDataGenerator for augmentation
# datagen = ImageDataGenerator(
#     rotation_range=0,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.1,
#     fill_mode='nearest')

# # Get a list of all files in the input folder
# files = os.listdir(input_folder)

# # Loop through each file in the input folder
# for file in files:
#     # Check if the file is an image
#     if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
#         # Load the image
#         image_path = os.path.join(input_folder, file)
#         image = np.array(Image.open(image_path))
#         image = image.reshape((1,) + image.shape)  # Reshape the image for the flow method

#         # Generate augmented images and save in the same folder
#         i = 0
#         for batch in datagen.flow(image, batch_size=1, save_to_dir=input_folder, save_prefix='aug', save_format='jpeg'):
#             i += 1
#             if i >= 10:  # Generate 10 augmented images for each original image
#                 break

#         print(f'Augmented and saved images for: {file}')

# print('Augmentation complete.')


Augmented and saved images for: small_image_1695482881_3_0.png
Augmented and saved images for: small_image_1695482881_3_1.png
Augmented and saved images for: small_image_1695482881_3_2.png
Augmented and saved images for: small_image_1695482881_3_3.png
Augmented and saved images for: small_image_1695482881_3_4.png
Augmented and saved images for: small_image_1695482881_3_5.png
Augmented and saved images for: small_image_1695482881_3_7.png
Augmented and saved images for: small_image_1695482881_3_8.png
Augmented and saved images for: small_image_1695482881_3_9.png
Augmentation complete.


In [7]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Set the input folder path
train_folder = r'C:\Users\Nilanjan Vyas\Documents\Minor_Project\TestFolder'

# Initialize the ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.1,
    fill_mode='nearest')

# Get a list of all subfolders in the train folder
subfolders = [f.path for f in os.scandir(train_folder) if f.is_dir()]

# Loop through each subfolder
for folder in subfolders:
    # Get a list of all files in the subfolder
    files = os.listdir(folder)

    # Loop through each file in the subfolder
    for file in files:
        # Check if the file is an image
        if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
            # Load the image
            image_path = os.path.join(folder, file)
            image = np.array(Image.open(image_path))
            image = image.reshape((1,) + image.shape)  # Reshape the image for the flow method

            # Generate augmented images and save in the same folder
            i = 0
            for batch in datagen.flow(image, batch_size=1, save_to_dir=folder, save_prefix='aug', save_format='jpeg'):
                i += 1
                if i >= 10:  # Generate 10 augmented images for each original image
                    break

            print(f'Augmented and saved images for: {file} in folder: {folder}')

print('Augmentation complete.')


Augmented and saved images for: small_image_1695482881_3_0.png in folder: C:\Users\Nilanjan Vyas\Documents\Minor_Project\TestFolder\ba
Augmented and saved images for: small_image_1695482881_3_1.png in folder: C:\Users\Nilanjan Vyas\Documents\Minor_Project\TestFolder\ba
Augmented and saved images for: small_image_1695482881_3_2.png in folder: C:\Users\Nilanjan Vyas\Documents\Minor_Project\TestFolder\ba
Augmented and saved images for: small_image_1695482881_3_3.png in folder: C:\Users\Nilanjan Vyas\Documents\Minor_Project\TestFolder\ba
Augmented and saved images for: small_image_1695482881_3_4.png in folder: C:\Users\Nilanjan Vyas\Documents\Minor_Project\TestFolder\ba
Augmented and saved images for: small_image_1695482881_3_5.png in folder: C:\Users\Nilanjan Vyas\Documents\Minor_Project\TestFolder\ba
Augmented and saved images for: small_image_1695482881_3_7.png in folder: C:\Users\Nilanjan Vyas\Documents\Minor_Project\TestFolder\ba
Augmented and saved images for: small_image_1695482881_